In [1]:
# Importing libraries
import numpy as np
import tensorflow as tf

import tensorflow_datasets as tfds

In [2]:
# Loading Data
mnist_dataset, mnist_info = tfds.load(name='mnist', with_info=True, as_supervised=True)

mnist_train, mnist_test = mnist_dataset['train'], mnist_dataset['test']

In [3]:
# Preprocessing
num_validation_samples = 0.1 * mnist_info.splits['train'].num_examples
num_validation_samples = tf.cast(num_validation_samples, tf.int64)

num_test_samples = mnist_info.splits['test'].num_examples
num_test_samples = tf.cast(num_test_samples, tf.int64)

In [4]:
# Scaling
def scale(image, label):
    image = tf.cast(image, tf.float32)
    image /= 255.
    return image, label

scaled_train_and_validation_data = mnist_train.map(scale)
test_data = mnist_test.map(scale)

In [5]:
# Shuffle data
BUFFER_SIZE = 10000
shuffled_train_and_validation_data = scaled_train_and_validation_data.shuffle(BUFFER_SIZE)

validation_data = shuffled_train_and_validation_data.take(num_validation_samples)
train_data = shuffled_train_and_validation_data.skip(num_validation_samples)

BATCH_SIZE = 128

train_data = train_data.batch(BATCH_SIZE)
validation_data = validation_data.batch(num_validation_samples)
test_data = test_data.batch(num_test_samples)

In [6]:
validation_inputs, validation_targets = next(iter(validation_data))

In [7]:
# Model
input_size = 784
output_size = 10
hidden_layer_size = 250

model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28,28,1)),
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
    tf.keras.layers.Dense(output_size, activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [8]:
NUM_EPOCHS = 5

model.fit(train_data, epochs=NUM_EPOCHS, validation_data=(validation_inputs, validation_targets), verbose=2)

Epoch 1/5
422/422 - 3s - loss: 0.2652 - accuracy: 0.9192 - val_loss: 0.1190 - val_accuracy: 0.9618 - 3s/epoch - 7ms/step
Epoch 2/5
422/422 - 2s - loss: 0.0993 - accuracy: 0.9691 - val_loss: 0.0773 - val_accuracy: 0.9762 - 2s/epoch - 5ms/step
Epoch 3/5
422/422 - 2s - loss: 0.0703 - accuracy: 0.9781 - val_loss: 0.0681 - val_accuracy: 0.9783 - 2s/epoch - 5ms/step
Epoch 4/5
422/422 - 2s - loss: 0.0546 - accuracy: 0.9831 - val_loss: 0.0623 - val_accuracy: 0.9808 - 2s/epoch - 4ms/step
Epoch 5/5
422/422 - 2s - loss: 0.0417 - accuracy: 0.9866 - val_loss: 0.0496 - val_accuracy: 0.9862 - 2s/epoch - 5ms/step


In [9]:
test_loss, test_accuracy = model.evaluate(test_data)

1/1 [==============================] - 0s 276ms/step - loss: 0.0799 - accuracy: 0.9781


In [15]:
print('Test loss: {0:.2f}. Test accuracy: {1:.2f}%'.format(test_loss, test_accuracy*100.))

Test loss: 0.08. Test accuracy: 97.81%
